In [1]:
import argparse
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display
from multiprocessing import cpu_count,Pool 
from joblib import Parallel, delayed
import psutil
from orderedset import OrderedSet

In [2]:
cores = cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want
print ("number of cores "+str(partitions))

def merge_type(data):
    type_set = data.RefinedType.split('_')
    type_set.sort()
    uni_type_set = OrderedSet(type_set)
    data.RefinedType='_'.join(uni_type_set)
    
    type_set = data.Type.split('_')
    type_set.sort()
    uni_type_set = OrderedSet(type_set)
    data.Type='_'.join(uni_type_set)
    return data
def parallelize(data, func,pool):
    data_split = np.array_split(data, partitions)
    data = pd.concat(pool.map(func, data_split))
    return data


number of cores 28


In [3]:
long_event = pd.read_hdf('../../long_events.h5',key='LN_300')

In [4]:
d = long_event.groupby('RefinedType').size().to_frame(name='name').sort_values(by='name',ascending=False)
print (d.to_latex())

\begin{tabular}{lr}
\toprule
{} &    name \\
RefinedType   &         \\
\midrule
Construction  &  113984 \\
rain          &   41668 \\
Event         &   32144 \\
snow          &   27723 \\
fog           &   20847 \\
Congestion    &   17314 \\
Incident      &   13099 \\
cold          &   12083 \\
storm         &     733 \\
Other         &     440 \\
Lane          &     253 \\
Accident      &     226 \\
precipitation &      72 \\
Broken        &      49 \\
hail          &      14 \\
\bottomrule
\end{tabular}



In [8]:
long_event_ = pd.read_hdf('../../long_events_.h5',key='LN_300_nc2')

In [9]:
#new_LN = long_event_.apply(merge_type,axis=1)

In [10]:

d = long_event_.groupby('RefinedType').size().to_frame(name='name').sort_values(by='name',ascending=False)
print (d.to_latex())


\begin{tabular}{lr}
\toprule
{} &   name \\
RefinedType                                        &        \\
\midrule
rain                                               &  38253 \\
snow                                               &  25820 \\
Construction                                       &  22373 \\
fog                                                &  19553 \\
Event                                              &  16753 \\
cold                                               &  11671 \\
Congestion                                         &   3206 \\
Incident                                           &   2381 \\
Construction\_Event                                 &   1332 \\
Construction\_rain                                  &    758 \\
storm                                              &    709 \\
Congestion\_Incident                                &    675 \\
Congestion\_Event                                   &    516 \\
Event\_rain                                         &    514 

In [36]:
L = new_LN.copy()

In [37]:
L['s'] = L.RefinedType.astype('str')
a = L[L.s.str.len()>46]
a

,EventId,Type,RefinedType_,StartTime,EndTime,LocationLat,LocationLng,Distance(mi),AirportCode,Number,Street,Side,City,County,State,ZipCode,RefinedType,duration,s
268950,T-10834367_T-11971398_W-369722_T-10834367_T-11...,T_W,fog-moderate,2017-11-18 14:00:00,2017-12-14 14:00:00,-158.587319,-14.965989,NaN,KBFI,NaN,NaN,NaN,Seattle,NaN,WA,NaN,Congestion_Construction_Event_Incident_fog_rain,37440.000000,Congestion_Construction_Event_Incident_fog_rain
1328137,T-11733624_T-11733625_T-11733626_T-11811857_T-...,T_W,rain-light,2017-11-07 11:29:36,2017-11-22 05:59:00,-85.412457,5.310967,NaN,KMDW,NaN,NaN,NaN,Chicago,NaN,IL,NaN,Congestion_Construction_Event_Incident_rain_snow,21269.400000,Congestion_Construction_Event_Incident_rain_snow
2011389,T-14444253_W-2766910_T-12912052_T-12912079_T-1...,T_W,cold-severe,2017-08-26 01:15:00,2017-09-02 23:00:00,-96.508017,23.893171,NaN,KTME,NaN,NaN,NaN,Katy,NaN,TX,NaN,Congestion_Construction_Incident_Lane_cold_rain,11385.000000,Congestion_Construction_Incident_Lane_cold_rain
2486477,T-493094_T-493095_T-493098_T-493100_T-493101_T...,T,Construction,2016-10-19 10:01:00,2016-10-25 00:14:02,-118.239095,34.087310,0.00,KCQT,462.0,N Avenue 63,R,Los Angeles,Los Angeles,CA,90042.0,Congestion_Construction_Event_Incident_Lane_Other,8053.033333,Congestion_Construction_Event_Incident_Lane_Other
2708282,T-716374_T-717970_T-717971_T-717972_T-717973_T...,T,Event-Short,2016-08-13 07:00:00,2016-08-15 16:01:00,-118.246502,34.079773,0.00,KCQT,1000.0,Dodger Stadium,L,Los Angeles,Los Angeles,CA,90012.0,Accident_Congestion_Construction_Event_Incident,3421.000000,Accident_Congestion_Construction_Event_Incident
3456219,T-1714128_T-1714129_T-1714357_T-1714960_T-1714...,T,Congestion-Moderate,2017-01-26 12:16:00,2017-01-27 07:00:00,-73.917766,40.760490,2.40,KNYC,NaN,George Washington Brg,R,New York,New York,NY,10033.0,Accident_Congestion_Construction_Event_Incident,1124.000000,Accident_Congestion_Construction_Event_Incident
9647119,T-8814449_T-8815915_T-8816255_T-8816256_T-8816...,T,Construction,2018-03-12 14:00:23,2018-03-17 09:30:00,-118.350329,34.141130,0.29,KBUR,NaN,N Buena Vista St,R,Burbank,Los Angeles,CA,91504.0,Accident_Congestion_Construction_Event_Incident,6929.616667,Accident_Congestion_Construction_Event_Incident
10761252,T-9898204_T-9898212_T-9936148_T-9936216_T-9936...,T,Construction,2018-02-22 10:39:19,2018-02-24 11:22:00,-87.689862,41.812615,0.20,KMDW,7901.0,S Avalon Ave,L,Chicago,Cook,IL,60619.0,Accident_Congestion_Construction_Event_Incident,2922.683333,Accident_Congestion_Construction_Event_Incident
11734851,T-10917280_T-10917281_T-10917282_T-10917283_T-...,T,Construction,2017-12-04 15:51:03,2017-12-17 07:59:00,-118.322609,34.160576,0.55,KBUR,2669.0,N San Fernando Blvd,L,Burbank,Los Angeles,CA,91504.0,Accident_Congestion_Construction_Event_Incident,18247.950000,Accident_Congestion_Construction_Event_Incident
13215518,T-12405891_T-12405894_T-12405937_T-12406303_T-...,T,Incident-Flow,2017-10-09 17:18:35,2017-10-11 06:17:00,-117.751232,33.833483,11.19,KSNA,NaN,Jamboree Rd,R,Tustin,Orange,CA,92782.0,Accident_Congestion_Construction_Event_Incident,2218.416667,Accident_Congestion_Construction_Event_Incident
